# Population Analysis by SQL Queries

## Import necessary libraries

In [86]:
#import Libraries
import pandas as pd
import os
from pandasql import sqldf

## Data Collection

In [87]:
#import files
df=pd.read_csv(r"C:\Users\HP-PC\Desktop\Projects\population\population.csv")
df.head()

,Rank,State,Capital,Population,% of Total Population,Males,Females,Sex Ratio,Literacy Rate (%),Rural Population,Urban Population,Area (km*km),Density (1/km*km),Decadal Growth (%)
0,1,Uttar Pradesh,Lucknow,"199,812,341",16.50,"104,480,510","95,331,831",912,67.68,"155,111,022","44,470,455","240,928",828,20.1%
1,2,Maharashtra,Mumbai,"112,374,333",9.28,"58,243,056","54,131,277",929,82.34,"61,545,441","50,827,531","307,713",365,16.0%
2,3,Bihar,Patna,"104,099,452",8.60,"54,278,157","49,821,295",918,61.80,"92,075,028","11,729,609","94,163","1,102",25.1%
3,4,West Bengal,Kolkata,"91,276,115",7.54,"46,809,027","44,467,088",950,76.26,"62,213,676","29,134,060","88,752","1,030",13.9%
4,5,Andhra Pradesh,Hyderabad,"84,580,777",6.99,"42,442,146","42,138,631",993,67.02,"56,361,702","28,219,075","275,045",308,10.98%


## Data Exploration

In [88]:
df.shape

(35, 14)

In [89]:
df.columns

Index(['Rank', 'State', 'Capital', 'Population', '% of Total Population',
       'Males', 'Females', 'Sex Ratio', 'Literacy Rate (%)',
       'Rural Population', 'Urban Population', 'Area (km*km)',
       'Density (1/km*km)', 'Decadal Growth (%)'],
      dtype='object')

### Check Data

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rank                   35 non-null     int64  
 1   State                  35 non-null     object 
 2   Capital                34 non-null     object 
 3   Population             35 non-null     object 
 4   % of Total Population  35 non-null     float64
 5   Males                  35 non-null     object 
 6   Females                35 non-null     object 
 7   Sex Ratio              35 non-null     object 
 8   Literacy Rate (%)      35 non-null     float64
 9   Rural Population       35 non-null     object 
 10  Urban Population       35 non-null     object 
 11  Area (km*km)           35 non-null     object 
 12  Density (1/km*km)      35 non-null     object 
 13  Decadal Growth (%)     35 non-null     object 
dtypes: float64(2), int64(1), object(11)
memory usage: 4.0+ KB


### Correct the data type

In [91]:
#create function to change the object data to float
def object_to_int(data):
    df[data]=df[data].str.replace(",","")
    df[data]=df[data].str.replace("%","")
    df[data]=df[data].astype('float')

#columns
columns=["Population","Males","Females","Sex Ratio","Rural Population","Urban Population","Area (km*km)","Density (1/km*km)"]

for col in columns:
    object_to_int(col)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rank                   35 non-null     int64  
 1   State                  35 non-null     object 
 2   Capital                34 non-null     object 
 3   Population             35 non-null     float64
 4   % of Total Population  35 non-null     float64
 5   Males                  35 non-null     float64
 6   Females                35 non-null     float64
 7   Sex Ratio              35 non-null     float64
 8   Literacy Rate (%)      35 non-null     float64
 9   Rural Population       35 non-null     float64
 10  Urban Population       35 non-null     float64
 11  Area (km*km)           35 non-null     float64
 12  Density (1/km*km)      35 non-null     float64
 13  Decadal Growth (%)     35 non-null     object 
dtypes: float64(10), int64(1), object(3)
memory usage: 4.0+ KB


### 1.	Retrieve State and Population for States with Population above 5 crore (50 million)

In [92]:
task="select State, Population from df where Population > 50000000"
sqldf(task,globals())

,State,Population
0,Uttar Pradesh,199812341.0
1,Maharashtra,112374333.0
2,Bihar,104099452.0
3,West Bengal,91276115.0
4,Andhra Pradesh,84580777.0
5,Madhya Pradesh,72626809.0
6,Tamil Nadu,72147030.0
7,Rajasthan,68548437.0
8,Karnataka,61095297.0
9,Gujarat,60439692.0


### 2.	Calculate the Average Literacy Rate and Average Density

In [93]:
task='SELECT AVG("Literacy Rate (%)") AS Avg_Literacy_Rate, AVG("Density (1/km*km)") AS Avg_Density FROM df'
sqldf(task,globals())

,Avg_Literacy_Rate,Avg_Density
0,77.940286,1091.885714


### 3.	Find the State with the highest Male to Female Ratio:

In [94]:
task='SELECT State, Males, Females, "Sex Ratio" FROM df ORDER BY "Sex Ratio" DESC LIMIT 1'
sqldf(task,globals())

,State,Males,Females,Sex Ratio
0,Kerala,16027412.0,17378649.0,1084.0


### 4.	Retrieve the Top 5 States with the Highest Urban Population Percentage

In [95]:
task='SELECT State, UrbanPopulationPercentage FROM (SELECT State, round("Urban Population" / ("Rural Population" + "Urban Population") * 100,2) AS UrbanPopulationPercentage FROM df) AS UrbanPopPercentages ORDER BY UrbanPopulationPercentage DESC LIMIT 5'
sqldf(task,globals())

,State,UrbanPopulationPercentage
0,Chandigarh,97.25
1,Delhi,93.18
2,Lakshadweep,78.08
3,Daman and Diu,75.16
4,Puducherry,68.31


### 5.	Calculate the Total Population and Area for Each Region (Rural and Urban) and Each State

In [96]:
task='SELECT State,SUM("Rural Population") AS Total_Rural_Population, SUM("Urban Population") AS Total_Urban_Population, SUM("Area (km*km)") AS Total_Area FROM df GROUP BY State'
sqldf(task,globals())

,State,Total_Rural_Population,Total_Urban_Population,Total_Area
0,Andaman and Nicobar Islands,244411.0,135533.0,8249.0
1,Andhra Pradesh,56361702.0,28219075.0,275045.0
2,Arunachal Pradesh,1069165.0,313446.0,83743.0
3,Assam,26780526.0,4388756.0,78438.0
4,Bihar,92075028.0,11729609.0,94163.0
5,Chandigarh,29004.0,1025682.0,114.0
6,Chhattisgarh,19603658.0,5936538.0,135191.0
7,Dadra and Nagar Haveli,183024.0,159829.0,491.0
8,Daman and Diu,60331.0,182580.0,112.0
9,Delhi,944727.0,12905780.0,1484.0


### 6.	Find the States with a Literacy Rate above the National Average Literacy Rate

In [97]:
task='SELECT State, "Literacy Rate (%)" FROM df WHERE "Literacy Rate (%)" > (SELECT AVG("Literacy Rate (%)") FROM df)'
sqldf(task,globals())

,State,Literacy Rate (%)
0,Maharashtra,82.34
1,Tamil Nadu,80.09
2,Gujarat,78.03
3,Kerala,94.00
4,Delhi,86.21
5,Uttarakhand,79.63
6,Himachal Pradesh,82.80
7,Tripura,87.22
8,Manipur,79.21
9,Nagaland,79.55


### 7.	Retrieve the Capital and Population for the 3 Most Populated States

In [98]:
task='SELECT Capital, Population FROM df ORDER BY Population DESC LIMIT 3'
sqldf(task,globals())

,Capital,Population
0,Lucknow,199812341.0
1,Mumbai,112374333.0
2,Patna,104099452.0


### 8.	Calculate the average population of states

In [99]:
#task='SELECT State, AVG(Population) OVER () AS avg_population FROM df'
task='SELECT State, AVG(Population) AS avg_population FROM df'
sqldf(task,globals())

,State,avg_population
0,Uttar Pradesh,3.459203e+07


### 9.	Rank states by population

In [100]:
task='SELECT Rank, State, Population, RANK() OVER (ORDER BY Population DESC) AS population_rank FROM df'
sqldf(task,globals())

,Rank,State,Population,population_rank
0,1,Uttar Pradesh,199812341.0,1
1,2,Maharashtra,112374333.0,2
2,3,Bihar,104099452.0,3
3,4,West Bengal,91276115.0,4
4,5,Andhra Pradesh,84580777.0,5
5,6,Madhya Pradesh,72626809.0,6
6,7,Tamil Nadu,72147030.0,7
7,8,Rajasthan,68548437.0,8
8,9,Karnataka,61095297.0,9
9,10,Gujarat,60439692.0,10


### 10.	Calculate the population share percentage for each state

In [101]:
task='SELECT State, Population, (Population / SUM(Population) OVER ()) * 100 AS population_share_percentage FROM df'
sqldf(task,globals())

,State,Population,population_share_percentage
0,Uttar Pradesh,199812341.0,16.503583
1,Maharashtra,112374333.0,9.281605
2,Bihar,104099452.0,8.598138
3,West Bengal,91276115.0,7.538989
4,Andhra Pradesh,84580777.0,6.985984
5,Madhya Pradesh,72626809.0,5.998642
6,Tamil Nadu,72147030.0,5.959014
7,Rajasthan,68548437.0,5.661787
8,Karnataka,61095297.0,5.046191
9,Gujarat,60439692.0,4.992042


### 11.	Calculate the cumulative population percentage using window function

In [102]:
task='SELECT State,Population,SUM(Population) OVER (ORDER BY Population DESC) AS cumulative_population,(SUM(Population) OVER (ORDER BY Population DESC) / SUM(Population) OVER ()) * 100 AS cumulative_population_percentage FROM df'
sqldf(task,globals())

,State,Population,cumulative_population,cumulative_population_percentage
0,Uttar Pradesh,199812341.0,1.998123e+08,16.503583
1,Maharashtra,112374333.0,3.121867e+08,25.785188
2,Bihar,104099452.0,4.162861e+08,34.383326
3,West Bengal,91276115.0,5.075622e+08,41.922315
4,Andhra Pradesh,84580777.0,5.921430e+08,48.908299
5,Madhya Pradesh,72626809.0,6.647698e+08,54.906941
6,Tamil Nadu,72147030.0,7.369169e+08,60.865955
7,Rajasthan,68548437.0,8.054653e+08,66.527741
8,Karnataka,61095297.0,8.665606e+08,71.573933
9,Gujarat,60439692.0,9.270003e+08,76.565974


### 12.	Calculate the difference in population from the previous state

In [103]:
task='select State, Population, lag(Population) over(order by rank) as "Previous Population",Population-lag(Population) over(order by rank) as "Population Difference" from df'
sqldf(task,globals())

,State,Population,Previous Population,Population Difference
0,Uttar Pradesh,199812341.0,NaN,NaN
1,Maharashtra,112374333.0,199812341.0,-87438008.0
2,Bihar,104099452.0,112374333.0,-8274881.0
3,West Bengal,91276115.0,104099452.0,-12823337.0
4,Andhra Pradesh,84580777.0,91276115.0,-6695338.0
5,Madhya Pradesh,72626809.0,84580777.0,-11953968.0
6,Tamil Nadu,72147030.0,72626809.0,-479779.0
7,Rajasthan,68548437.0,72147030.0,-3598593.0
8,Karnataka,61095297.0,68548437.0,-7453140.0
9,Gujarat,60439692.0,61095297.0,-655605.0
